**YOLO**

In [2]:
from ultralytics import YOLO

# 加载已经修复好的模型
model = YOLO(r'H:\TCV_PROJECT\best.pt')

print("--- Evaluating Self-Captured Dataset ---")
metrics_self = model.val(data='Public.yaml')


print(f"Precision: {metrics_self.results_dict['metrics/precision(B)']:.4f}")
print(f"Recall: {metrics_self.results_dict['metrics/recall(B)']:.4f}")
print(f"mAP50: {metrics_self.results_dict['metrics/mAP50(B)']:.4f}") 

--- Evaluating Self-Captured Dataset ---
Ultralytics 8.4.9  Python-3.10.11 torch-2.11.0.dev20260130+cu130 CUDA:0 (NVIDIA GeForce RTX 5060, 8150MiB)
YOLO11n summary (fused): 101 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 233.380.1 MB/s, size: 61.7 KB)
val: Scanning H:\TCV_PROJECT\dataset\kagel img\labels\val... 1467 images, 4 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1467/1467 3.1Kit/s 0.5s0.1s
val: New cache created: H:\TCV_PROJECT\dataset\kagel img\labels\val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 92/92 8.5it/s 10.8s0.1s
                   all       1467       4583      0.189      0.198     0.0861     0.0528
               plastic        474       1547      0.301     0.0922     0.0973     0.0588
                 paper        166        363     0.0675      0.433     0.0597     0.0352
                 metal        592       1693      0.265      0.216     

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report

# 1. 基础配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_val_path = 'H:/TCV_PROJECT/dataset/self capture img/resnet_val_format'

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 2. 加载模型架构
model_resnet = models.resnet50(weights=None)
model_resnet.fc = nn.Linear(model_resnet.fc.in_features, 4)

# 【关键：如果你有权重文件，取消下面这行的注释】
# model_resnet.load_state_dict(torch.load(r'H:\TCV_PROJECT\resnet_best.pth'))

model_resnet = model_resnet.to(device).eval()

# 3. 运行评估
test_set = datasets.ImageFolder(resnet_val_path, transform=data_transforms)
loader = DataLoader(test_set, batch_size=32, shuffle=False)

y_true, y_pred = [], []
with torch.no_grad():
    for inputs, labels in loader:
        outputs = model_resnet(inputs.to(device))
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds.cpu().numpy())

print("\n[ResNet50] Results:")
print(classification_report(y_true, y_pred, target_names=test_set.classes)) #

**ResNet50**

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report

# 1. 基础配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_val_path = 'H:/TCV_PROJECT/dataset/self capture img/resnet_val_format'

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 2. 加载模型架构
model_resnet = models.resnet50(weights=None)
model_resnet.fc = nn.Linear(model_resnet.fc.in_features, 4)

# 【关键：如果你有权重文件，取消下面这行的注释】
# model_resnet.load_state_dict(torch.load(r'H:\TCV_PROJECT\resnet_best.pth'))

model_resnet = model_resnet.to(device).eval()

# 3. 运行评估
test_set = datasets.ImageFolder(resnet_val_path, transform=data_transforms)
loader = DataLoader(test_set, batch_size=32, shuffle=False)

y_true, y_pred = [], []
with torch.no_grad():
    for inputs, labels in loader:
        outputs = model_resnet(inputs.to(device))
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds.cpu().numpy())

print("\n[ResNet50] Results:")
print(classification_report(y_true, y_pred, target_names=test_set.classes)) #

**HOG + SVM**

In [ ]:
import cv2
import os
import numpy as np
import joblib
from sklearn.metrics import classification_report

# 路径配置
test_path = 'H:/TCV_PROJECT/dataset/self capture img/resnet_val_format'
categories = ['glass', 'metal', 'paper', 'plastic']

# 提取 HOG 特征
def extract_hog(folder_path):
    features, labels = [], []
    hog = cv2.HOGDescriptor()
    for idx, cat in enumerate(categories):
        p = os.path.join(folder_path, cat)
        for img_n in os.listdir(p):
            img = cv2.imread(os.path.join(p, img_n))
            if img is None: continue
            img = cv2.resize(img, (128, 128))
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            features.append(hog.compute(gray).flatten())
            labels.append(idx)
    return np.array(features), np.array(labels)

